# Processing data from GitHub

In [1]:
import pydriller
import os
import json
from dpu_utils.utils import save_jsonl_gz, load_json_gz, RichPath
import numpy as np

from collections import namedtuple, defaultdict
import json
from tqdm import tqdm
from joblib import Parallel, delayed, cpu_count
%load_ext autoreload
%autoreload 2

os.chdir('D:\github_data')

commit_data_dir = 'preprocessed_data_no_unchanged_short'
os.makedirs(commit_data_dir, exist_ok=True)

### Объединим данные со всех репозиториев в один DataFrame

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import gc

import gzip

pd.options.display.float_format = '{:.2f}'.format

dfs = []

large_files = []

for filename in tqdm(os.listdir(commit_data_dir)):
    try:
        df = pd.read_json(os.path.join(commit_data_dir, filename), orient='records')
        df['repo'] = filename.split('.json')[0]
        dfs.append(df)
    except ValueError:
        print('ValueError with', filename)
        large_files.append(filename)
    except MemoryError:
        print('MemoryError with', filename)
        large_files.append(filename)

100%|████████████████████████████████████████████████████████████████████████████████| 995/995 [01:43<00:00,  9.59it/s]


In [3]:
df = pd.concat(dfs, ignore_index=True, axis=0)
df.author = df.author.apply(lambda x: tuple(x))
df.head()

,author,date,message,diff,num_mods,diff_len,message_len,repo
0,"(Arnaud Barisain Monrose, DREAMTEAM69@gmail.com)",2010-06-24 22:59:02,Fixed 2 . 1 compatibility,[. DS _ Store \n Binary files a / . DS _ Store...,12,764,5,abarisain_dmix
1,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 03:14:53,Updated project so it uses 2 . 2 apis,[MPDroid / AndroidManifest . xml \n + < receiv...,3,167,9,abarisain_dmix
2,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 03:16:28,Fixed text shadow size and cover art DPI,[MPDroid / res / layout / main . xml \n - andr...,2,972,8,abarisain_dmix
3,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 03:19:09,Added headphone unplug & remote control support,[MPDroid / src / com / arkanta / mpdroid / Str...,1,1402,7,abarisain_dmix
4,"(dreamteam69, dreamteam69@gmail.com)",2010-08-07 14:50:35,Fixed 2 . 1 compatibility,[MPDroid / default . properties \n - target = ...,3,1197,5,abarisain_dmix


In [4]:
len(df)

1692920

### Статистика по коммитам для каждого автора

In [5]:
author_df = df.groupby(by='author')['date'].count()
author_df.describe()

count   58571.00
mean       28.90
std       172.61
min         1.00
25%         1.00
50%         2.00
75%         7.00
max     12513.00
Name: date, dtype: float64

Всего есть ~59k уникальных авторов, причем тех, у кого всего один коммит, достаточно много. Посмотрим на конкретное соотношение:

In [6]:
print(f'% of authors with only 1 commit: {100 * len(author_df.loc[author_df == 1]) / len(author_df):.2f}')

% of authors with only 1 commit: 41.67


In [7]:
print(len(df) - len(df.loc[df['author'].isin(author_df.index[author_df > 1].tolist())]))

24405


Теоретически, таких авторов можно отбросить (это лишит нас всего 24k примеров), но можно и оставить, чтобы лучше обрабатывать случаи, когда истории нет.

### Статистика по количеству токенов в диффах

Здесь токенизация проводится просто разделением по пробелам, и в данной версии датасета отброшены все примеры более чем с 2048 токенами.

In [8]:
df['diff_len'].describe()

count   1692920.00
mean        531.18
std         512.13
min          12.00
25%         127.00
50%         332.00
75%         796.00
max        2048.00
Name: diff_len, dtype: float64

In [9]:
print(f"% of examples with # of tokens in diff <= 512: {100 * len(df.loc[df['diff_len'] <= 512]) / len(df):.2f}")

% of examples with # of tokens in diff <= 512: 62.34


### Статистика по количеству токенов в сообщениях

In [10]:
df['message_len'].describe()

count   1692920.00
mean         26.42
std          24.63
min           5.00
25%           9.00
50%          16.00
75%          41.00
max         158.00
Name: message_len, dtype: float64

### Сохраним данные в формате открытого датасета

Открытый датасет был разбит на trait и test части, в каждой из них хранилось по три файла: с сообщениями, с диффами, и с id репозитория. Сделаем примерно то же самое, но с id автора, а еще отсортируем по времени.

**UPD: мне кажется более удобным использовать единый .csv файл.**

In [11]:
# отсортируем датафрейм так, чтобы у каждого автора коммиты шли в соответствии с датой, от самых ранних к самым поздним
df.sort_values(by=['author', 'date'], inplace=True)

# заменим авторов на что-то вроде id, чтобы было проще
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['author'] = df['author'].apply(lambda x: x[0] + '<SEP>' + x[1]) # label encoder doesn't work with tuples
df['author'] = le.fit_transform(df['author'])

# сделаем из диффов-списков диффы-строки
df['diff'] = df['diff'].apply(lambda x: ' '.join(x))

Вынесем в train, val и test разные репозитории.

In [12]:
from sklearn.model_selection import train_test_split

repos = df['repo'].unique()

train_repos, test_repos = train_test_split(repos, test_size=0.01, shuffle=True, random_state=555)
train_repos, val_repos = train_test_split(train_repos, test_size=0.01, random_state=555)

In [13]:
print('Train:', len(train_repos))
print('Val:', len(val_repos))
print('Test:', len(test_repos))

Train: 975
Val: 10
Test: 10


In [14]:
test_df = df.loc[df['repo'].isin(test_repos)]
test_df.shape

(14144, 8)

In [15]:
val_df = df.loc[df['repo'].isin(val_repos)]
val_df.shape

(14127, 8)

In [16]:
train_df = df.loc[df['repo'].isin(train_repos)]
train_df.shape

(1664649, 8)

Теперь дополнительно проверим, что в train не встречаются авторы из val и test.

In [17]:
test_authors = test_df['author'].unique()
val_authors = val_df['author'].unique()

print('Intersection between val and test:')
print(np.intersect1d(test_authors, val_authors))

test_and_val_authors = list(test_authors) +  list(val_authors)

Intersection between val and test:
[42321 46225 51044 52049]


In [18]:
train_df = train_df.loc[~train_df['author'].isin(test_and_val_authors)]
train_df.shape

(1594188, 8)

In [19]:
train_df.head()

,author,date,message,diff,num_mods,diff_len,message_len,repo
1473662,97,2014-05-11 21:19:07,And re - add renamed file . . .,src / main / java / org / spongepowered / api ...,3,122,9,spongepowered_spongeapi
305788,99,2001-10-24 09:07:36,"Added method getText ( page , version ) , getH...",src / com / ecyrd / jspwiki / WikiEngine . jav...,1,386,78,apache_jspwiki
305849,99,2001-11-23 15:10:24,Initial version of tag library object for quer...,new file \n src / com / ecyrd / jspwiki / tags...,1,1467,47,apache_jspwiki
305850,99,2001-11-23 15:24:39,Modified to store WikiLinks encountered during...,src / com / ecyrd / jspwiki / TranslatorReader...,1,341,53,apache_jspwiki
305851,99,2001-11-23 15:26:05,"Modified to create a static ReferenceManager ,...",src / com / ecyrd / jspwiki / WikiEngine . jav...,1,791,80,apache_jspwiki


In [22]:
train_df['author'].nunique()

57116

In [23]:
val_df['author'].nunique()

1022

In [24]:
test_df['author'].nunique()

437

In [21]:
train_df[['diff', 'message', 'author']].to_csv('train.csv', header=None, index=None)
val_df[['diff', 'message', 'author']].to_csv('val.csv', header=None, index=None)
test_df[['diff', 'message', 'author']].to_csv('test.csv', header=None, index=None)

Код, если все-таки захочется сохранить датасет в виде трех отдельных файлов:

In [50]:
def save_df(df, folder):
    msgs = list(item[1]+'\n' for item in df['message'].items())
    with open(os.path.join(folder, 'msg.txt'), 'w', encoding='utf-8') as target:
        target.writelines(msgs)
        
    authors = list(str(item[1])+'\n' for item in df['author'].items())
    with open(os.path.join(folder, 'ds.txt'), 'w', encoding='utf-8') as target:
        target.writelines(authors)
        
    diffs = list(item[1]+'\n' for item in df['diff'].items())
    with open(os.path.join(folder, 'diff.txt'), 'w', encoding='utf-8') as target:
        target.writelines(diffs)